In [82]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

import seaborn as sns
%matplotlib inline

Let's take a look at the data we have

In [83]:
df = pd.read_csv('./stack-overflow-developer-survey-2021/survey_results_public.csv')
df.head().T

,0,1,2,3,4
ResponseId,1,2,3,4,5
MainBranch,I am a developer by profession,I am a student who is learning to code,"I am not primarily a developer, but I write co...",I am a developer by profession,I am a developer by profession
Employment,"Independent contractor, freelancer, or self-em...","Student, full-time","Student, full-time",Employed full-time,"Independent contractor, freelancer, or self-em..."
Country,Slovakia,Netherlands,Russian Federation,Austria,United Kingdom of Great Britain and Northern I...
US_State,NaN,NaN,NaN,NaN,NaN
UK_Country,NaN,NaN,NaN,NaN,England
EdLevel,"Secondary school (e.g. American high school, G...","Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)"
Age1stCode,18 - 24 years,11 - 17 years,11 - 17 years,11 - 17 years,5 - 10 years
LearnCode,Coding Bootcamp;Other online resources (ex: vi...,"Other online resources (ex: videos, blogs, etc...","Other online resources (ex: videos, blogs, etc...",NaN,Friend or family member
YearsCode,NaN,7,NaN,NaN,17


In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83439 entries, 0 to 83438
Data columns (total 48 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ResponseId                    83439 non-null  int64  
 1   MainBranch                    83439 non-null  object 
 2   Employment                    83323 non-null  object 
 3   Country                       83439 non-null  object 
 4   US_State                      14920 non-null  object 
 5   UK_Country                    4418 non-null   object 
 6   EdLevel                       83126 non-null  object 
 7   Age1stCode                    83243 non-null  object 
 8   LearnCode                     82963 non-null  object 
 9   YearsCode                     81641 non-null  object 
 10  YearsCodePro                  61216 non-null  object 
 11  DevType                       66484 non-null  object 
 12  OrgSize                       60726 non-null  object 
 13  C

Our main task is to predict an expected salary. This problem is non-trivial since we have just a **CompTotal** value differing in currency (**Currency**) and frequency (**CompFreq**).

We should decide:

1. In which currency we want to predict the salary. USD seems most reasonable. We can convert other currencies or just drop them.
2. For which period? (monthly, daily, hourly rate, annual). I think, annual is the best choice.
3. It can be also a classification problem: for example, is the annual salary higher than 70k?
4. Also classification problem: predict highest paid programming language.
5. Any other thoughts are very welcome!

## Preprocessing

First of all, let's drop all rows that doesn't contain salary info and column with useless id.

In [85]:
df.dropna(subset=['CompTotal'], inplace=True)
df.drop('ResponseId', axis=1, inplace=True)

We've got a lot of columns, let's leave only some of them (that seem most important). First of all, we split the columns to numerical and categorical and have a look how many values each category has. If this number is too high we're not gonna use it in our initial analysis.

In [86]:
num_cols = df.select_dtypes(include=[np.number]).columns
cat_cols = df.select_dtypes(include=['object']).columns

In [87]:
for col in cat_cols:
    print(col, len(df[col].unique()))

MainBranch 2
Employment 6
Country 171
US_State 53
UK_Country 5
EdLevel 10
Age1stCode 10
LearnCode 434
YearsCode 53
YearsCodePro 53
DevType 6536
OrgSize 11
Currency 139
CompFreq 4
LanguageHaveWorkedWith 17819
LanguageWantToWorkWith 16815
DatabaseHaveWorkedWith 2598
DatabaseWantToWorkWith 2474
PlatformHaveWorkedWith 105
PlatformWantToWorkWith 122
WebframeHaveWorkedWith 4159
WebframeWantToWorkWith 3497
MiscTechHaveWorkedWith 996
MiscTechWantToWorkWith 1219
ToolsTechHaveWorkedWith 714
ToolsTechWantToWorkWith 1115
NEWCollabToolsHaveWorkedWith 7706
NEWCollabToolsWantToWorkWith 4483
OpSys 7
NEWStuck 1066
NEWSOSites 9
SOVisitFreq 6
SOAccount 4
SOPartFreq 7
SOComm 7
NEWOtherComms 3
Age 9
Gender 17
Trans 5
Sexuality 30
Ethnicity 266
Accessibility 27
MentalHealth 34
SurveyLength 4
SurveyEase 4


We're gonna drop this columns

In [88]:
columns_to_drop = [col for col in cat_cols if len(df[col].unique()) > 15 and col != 'Currency']
columns_to_drop

['Country',
 'US_State',
 'LearnCode',
 'YearsCode',
 'YearsCodePro',
 'DevType',
 'LanguageHaveWorkedWith',
 'LanguageWantToWorkWith',
 'DatabaseHaveWorkedWith',
 'DatabaseWantToWorkWith',
 'PlatformHaveWorkedWith',
 'PlatformWantToWorkWith',
 'WebframeHaveWorkedWith',
 'WebframeWantToWorkWith',
 'MiscTechHaveWorkedWith',
 'MiscTechWantToWorkWith',
 'ToolsTechHaveWorkedWith',
 'ToolsTechWantToWorkWith',
 'NEWCollabToolsHaveWorkedWith',
 'NEWCollabToolsWantToWorkWith',
 'NEWStuck',
 'Gender',
 'Sexuality',
 'Ethnicity',
 'Accessibility',
 'MentalHealth']

In [89]:
df.drop(columns_to_drop, axis=1, inplace=True)

This is a cleaned dataframe.

In [90]:
df.head().T

,0,9,11,12,16
MainBranch,I am a developer by profession,I am a developer by profession,I am a developer by profession,I am a developer by profession,I am a developer by profession
Employment,"Independent contractor, freelancer, or self-em...",Employed full-time,Employed full-time,Employed full-time,Employed full-time
UK_Country,NaN,NaN,NaN,NaN,NaN
EdLevel,"Secondary school (e.g. American high school, G...","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)"
Age1stCode,18 - 24 years,11 - 17 years,11 - 17 years,11 - 17 years,5 - 10 years
OrgSize,20 to 99 employees,10 to 19 employees,10 to 19 employees,"1,000 to 4,999 employees",100 to 499 employees
Currency,EUR European Euro,SEK\tSwedish krona,EUR European Euro,EUR European Euro,TRY\tTurkish lira
CompTotal,4800.0,42000.0,43000.0,71500.0,9000.0
CompFreq,Monthly,Monthly,Yearly,Yearly,Monthly
OpSys,MacOS,Linux-based,Linux-based,Linux-based,Windows


TODO: **convert all salaries to USD instead filtering all other out**

In [92]:
df['Currency'].value_counts()

USD\tUnited States dollar    11760
EUR European Euro            11143
INR\tIndian rupee             3782
GBP\tPound sterling           2976
CAD\tCanadian dollar          1704
                             ...  
CDF\tCongolese franc             1
none\tCook Islands dollar        1
GNF\tGuinean franc               1
TOP\tTongan pa’anga              1
BIF\tBurundi franc               1
Name: Currency, Length: 139, dtype: int64

In [93]:
df = df[df['Currency'] == 'USD\tUnited States dollar']

In [94]:
df['CompFreq'].value_counts()

Yearly     9441
Monthly    1676
Weekly      600
Name: CompFreq, dtype: int64

In [95]:
df['salary'] = df['CompTotal'].mask(df['CompFreq'] == 'Monthly', df['CompTotal'] * 12)
df['salary'] = df['CompTotal'].mask(df['CompFreq'] == 'Weekly', df['CompTotal'] * 52)

In [96]:
df.head().T

,34,36,37,62,64
MainBranch,I am a developer by profession,I am a developer by profession,I am a developer by profession,"I am not primarily a developer, but I write co...","I am not primarily a developer, but I write co..."
Employment,Employed full-time,Employed full-time,Employed full-time,Employed full-time,"Independent contractor, freelancer, or self-em..."
UK_Country,NaN,NaN,NaN,NaN,NaN
EdLevel,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Some college/university study without earning ...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)"
Age1stCode,11 - 17 years,11 - 17 years,18 - 24 years,11 - 17 years,11 - 17 years
OrgSize,2 to 9 employees,20 to 99 employees,"1,000 to 4,999 employees","10,000 or more employees",2 to 9 employees
Currency,USD\tUnited States dollar,USD\tUnited States dollar,USD\tUnited States dollar,USD\tUnited States dollar,USD\tUnited States dollar
CompTotal,2000.0,103000.0,300000.0,130000.0,500000.0
CompFreq,Monthly,Yearly,Yearly,Yearly,Yearly
OpSys,Windows,Linux-based,MacOS,Windows,Linux-based


In [ ]:
def calc_annual_salary(row):
    row['CompFreq'] == 'Yearly'

In [73]:
df.apply(lambda row: row['Cop'], axis=1)

0                  EUR European Euro
9                 SEK\tSwedish krona
11                 EUR European Euro
12                 EUR European Euro
16                 TRY\tTurkish lira
                    ...             
83434      USD\tUnited States dollar
83435    XOF\tWest African CFA franc
83436      USD\tUnited States dollar
83437           CAD\tCanadian dollar
83438            BRL\tBrazilian real
Length: 47183, dtype: object

Suppose we want to predict the salaries. Let's clean the data at first.

In [43]:
def clean_data(df: pd.DataFrame):
    X = df.drop('CompTotal', axis=1)
    y = df['CompTotal']
    
    num_cols = X.select_dtypes(include=[np.number]).columns
    for col in num_cols:
        X[col] = X[col].fillna(X[col].median())
    
    cat_cols = X.select_dtypes(include=['object']).columns
    for col in  cat_cols:
        X = pd.concat([X.drop(col, axis=1),
                       pd.get_dummies(X[col], prefix=col, prefix_sep='_', drop_first=True)],
                       axis=1)
    
    return X, y
    
X, y = clean_data(df)

In [44]:
X

,ResponseId,ConvertedCompYearly,"MainBranch_I am not primarily a developer, but I write code sometimes as part of my work",Employment_Employed part-time,Employment_I prefer not to say,"Employment_Independent contractor, freelancer, or self-employed",Employment_Retired,UK_Country_Northern Ireland,UK_Country_Scotland,UK_Country_Wales,...,Age_65 years or older,Age_Prefer not to say,Age_Under 18 years old,"Trans_Or, in your own words:",Trans_Prefer not to say,Trans_Yes,SurveyLength_Too long,SurveyLength_Too short,SurveyEase_Easy,SurveyEase_Neither easy nor difficult
0,1,62268.0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9,10,51552.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
11,12,46482.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
12,13,77290.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
16,17,17748.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83434,83435,160500.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
83435,83436,3960.0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
83436,83437,90000.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
83437,83438,816816.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [15]:
df['Currency'].value_counts()

USD\tUnited States dollar      14363
EUR European Euro              13932
INR\tIndian rupee               6427
GBP\tPound sterling             3547
CAD\tCanadian dollar            2108
                               ...  
PGK\tPapua New Guinean kina        1
CDF\tCongolese franc               1
none\tCook Islands dollar          1
TOP\tTongan pa’anga                1
ERN\tEritrean nakfa                1
Name: Currency, Length: 146, dtype: int64